# __생성형 모델로 다양한 기능 수행하기__

- 튜토리얼 난이도: ★☆☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_generate/generate_gpt.ipynb   

## 튜토리얼 소개
 
<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 성형외과 VOC 데이터 세트를 사용합니다. 각 VOC는 고객과 상담사가 나눈 상담 내용을 담고 있으며 tx와 rx로 화자가 구분되어 있습니다.
</div>
    
ThanoSQL을 사용하여 VOC 데이터를 입력하고 다양한 생성형 기능들을 수행해 봅니다.

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/1.5/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET healthcare_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY healthcare_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/healthcare_data/healthcare_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

In [4]:
%%thanosql
SELECT *
FROM healthcare_test
LIMIT 5

,cleaned_text
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,..."
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>healthcare_test</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>cleaned_text: VOC 전사 텍스트</li>
    </ul>
</div>

## __2. FAQ 생성__

In [5]:
%%thanosql
GENERATE USING GPT
OPTIONS (
    task = 'faq', 
    text_col='cleaned_text',
    result_col='faq_result'
    )
AS 
SELECT *
FROM healthcare_test

,cleaned_text,faq_result,faq_result_n_trial
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,...","[{'Q': '성형외과 상담을 어떻게 할 수 있나요?', 'A': '저희 실시간 비...",1
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...,"[{'Q': '재수술일 경우에는 추가비용이 발생하나요?', 'A': '네, 재수술일...",1
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...,"[{'Q': '안면윤곽 수술을 하려고 하는데, 당일 검사 가능한가요?', 'A': ...",1
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...,"[{'Q': '검사와 수술 가능한 날짜를 알고 싶어요.', 'A': '이번 주 안으...",1
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...,"[{'Q': '가격은 어느 정도인가요?', 'A': '수술에 따라 다르지만, 사각턱...",1
5,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 안녕하세요.\...,"[{'Q': '더블로 리프팅이 무엇인가요?', 'A': '더블로 리프팅은 얼굴의 피...",1
6,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 아큐스컬프...,"[{'Q': '아큐스컬프랑 복부성형술의 차이점은 뭐예요?', 'A': '아큐스컬프는...",1
7,tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 여보세요? 네 네\nt...,"[{'Q': '안녕하세요 [ㅇㅇㅇ]님 맞으세요?', 'A': '네 여보세요? 네 네...",1
8,tx: 네 여보세요? 네네\nrx: 여보세요 네 검사실인데요.\ntx: 네네\...,"[{'Q': '이 분이 양성인 것을 알게 되면 어떻게 해야 하나요?', 'A': '...",1
9,tx: 아 하는 경우들도 있거든요\nrx: 그거는 알고 계셔서 병원에서 하시는거...,"[{'Q': '병원에서 어떤 경우에 검사를 하나요?', 'A': '병원에서 검사를 ...",1


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GENERATE USING</strong>" 쿼리 구문을 사용하여 <strong>GPT</strong> 모델을 생성에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"task": 실행할 작업의 종류</li>
            <li>"text_col": 생성에 사용할 텍스트를 담은 컬럼의 이름</li>
            <li>"result_col": 데이터 테이블에서 생성 결과를 담을 컬럼 이름</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 키워드 추출__

In [6]:
%%thanosql
GENERATE USING GPT
OPTIONS (
    task = 'keywords', 
    text_col='cleaned_text',
    result_col='keywords_result'
    )
AS 
SELECT *
FROM healthcare_test

,cleaned_text,keywords_result,keywords_result_n_trial
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,...","['안녕하세요', '상담', '비용', '수술', '주름']",1
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...,"['코 끝', '재수술', '추가비용', '기본가', '할인']",1
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...,"['사랑합니다', '성형외과', '안면윤곽', '검사', '전화']",1
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...,"['성형외과', '안면 윤곽', '수술 가능한 날짜', '상담', '예약']",1
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...,"['금액', '가격', '수술', '사각턱', '비용']",1
5,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 안녕하세요.\...,"['사랑합니다', '성형외과', '더블로 리프팅', '필러', '시술']",1
6,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 아큐스컬프...,"['아큐스컬프랑', '복부성형술', '복부', '지방흡입', '상담 비용']",1
7,tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 여보세요? 네 네\nt...,"['코 재수술', '윤곽', '광대', '시크릿광대', '비용']",1
8,tx: 네 여보세요? 네네\nrx: 여보세요 네 검사실인데요.\ntx: 네네\...,"['여보세요', '검사실', '수술전 검사', 'hiv 양성', '보건환경연구원']",1
9,tx: 아 하는 경우들도 있거든요\nrx: 그거는 알고 계셔서 병원에서 하시는거...,"['병원', '확인', '검사', '양성', '결과']",1


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GENERATE USING</strong>" 쿼리 구문을 사용하여 <strong>GPT</strong> 모델을 생성에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"task": 실행할 작업의 종류</li>
            <li>"text_col": 생성에 사용할 텍스트를 담은 컬럼의 이름</li>
            <li>"result_col": 데이터 테이블에서 생성 결과를 담을 컬럼 이름</li>
        </ul>
        </li>
    </ul>
</div>

## __4. 서비스 평가__

In [7]:
%%thanosql
GENERATE USING GPT
OPTIONS (
    task = 'service_evaluation', 
    text_col='cleaned_text',
    result_col='service_evaluation_result'
    )
AS 
SELECT *
FROM healthcare_test

,cleaned_text,service_evaluation_result,service_evaluation_result_n_trial
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,...","{'score': 4, 'explanation': '서비스 점수는 4점입니다. 상담...",1
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...,"{'score': 4, 'explanation': '서비스 점수는 4점입니다. 상담...",1
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...,"{'score': 4, 'explanation': '서비스 점수는 4점입니다. 이유...",1
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...,"{'score': 4, 'explanation': '서비스 점수는 4입니다. 상담원...",1
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...,"{'score': 4, 'explanation': '서비스 평가 점수는 4입니다. ...",1
5,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 안녕하세요.\...,"{'score': 4, 'explanation': '서비스 점수는 4입니다. 이유는...",1
6,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 아큐스컬프...,"{'score': 4, 'explanation': '서비스 평가 점수는 4입니다. ...",1
7,tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 여보세요? 네 네\nt...,"{'score': 4, 'explanation': '서비스 점수는 4입니다. 이유는...",1
8,tx: 네 여보세요? 네네\nrx: 여보세요 네 검사실인데요.\ntx: 네네\...,"{'score': 5, 'explanation': '이용자와의 대화에서 검사와 관련...",1
9,tx: 아 하는 경우들도 있거든요\nrx: 그거는 알고 계셔서 병원에서 하시는거...,"{'score': 4, 'explanation': '병원 직원은 친절하고 상세한 설...",1


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GENERATE USING</strong>" 쿼리 구문을 사용하여 <strong>GPT</strong> 모델을 생성에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"task": 실행할 작업의 종류</li>
            <li>"text_col": 생성에 사용할 텍스트를 담은 컬럼의 이름</li>
            <li>"result_col": 데이터 테이블에서 생성 결과를 담을 컬럼 이름</li>
        </ul>
        </li>
    </ul>
</div>

## __5. 내용 요약__

In [8]:
%%thanosql
GENERATE USING GPT
OPTIONS (
    task = 'summary', 
    text_col='cleaned_text',
    result_col='summary_result'
    )
AS 
SELECT *
FROM healthcare_test

,cleaned_text,summary_result,summary_result_n_trial
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,...","이 대화는 성형외과 상담에 관한 것으로, 하안검 수술을 통해 눈 밑 주름을 개선할 ...",1
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...,이 대화는 코 수술에 대한 상담 내용입니다. 환자는 재수술에 대한 추가비용이 발생할...,1
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...,사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다. 네 저기요 제가 안면윤곽 쪽으로 ...,1
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...,"사랑합니다 성형외과입니다. 안면 윤곽과 사각턱, 피질절제 등을 포함한 수술 가능한 ...",1
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...,"수술 비용에 대한 대화가 진행되고, 추가 비용이 있을 수 있다는 언급이 있었다. 또...",1
5,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 안녕하세요.\...,사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]에서는 더블로 리프팅 시술을 제공합니다. ...,1
6,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 아큐스컬프...,아큐스컬프와 복부성형술의 차이점에 대해 물어보는 대화입니다. 아큐스컬프는 얼굴 지방...,1
7,tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 여보세요? 네 네\nt...,이 대화는 성형외과 상담 내용을 다루고 있습니다. 상담 내용으로는 윤곽과 코 재수술...,1
8,tx: 네 여보세요? 네네\nrx: 여보세요 네 검사실인데요.\ntx: 네네\...,"이 대화는 HIV 양성 판정을 받은 환자의 검사 결과를 확인하고, 보건환경연구원에 ...",1
9,tx: 아 하는 경우들도 있거든요\nrx: 그거는 알고 계셔서 병원에서 하시는거...,"병원에서 양성으로 나온 결과를 보건환경연구원에 확인 받아야 하며, 위양성일 가능성을...",1


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GENERATE USING</strong>" 쿼리 구문을 사용하여 <strong>GPT</strong> 모델을 생성에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"task": 실행할 작업의 종류</li>
            <li>"text_col": 생성에 사용할 텍스트를 담은 컬럼의 이름</li>
            <li>"result_col": 데이터 테이블에서 생성 결과를 담을 컬럼 이름</li>
        </ul>
        </li>
    </ul>
</div>

## __6. 감정 분석__

In [9]:
%%thanosql
GENERATE USING GPT
OPTIONS (
    task = 'sentiment_analysis', 
    text_col='cleaned_text',
    result_col='sentiment_analysis_result'
    )
AS 
SELECT *
FROM healthcare_test

,cleaned_text,sentiment_analysis_result,sentiment_analysis_result_n_trial
0,"tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 네,...","{'sentiment': 'Positive', 'confidence_score': ...",1
1,tx: 아 그러세요?\nrx: 네\ntx: 그러시면 코 끝만 생각하시는거세요?...,"{'sentiment': 'Neutral', 'confidence_score': 0...",1
2,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 네 저기요 제...,"{'sentiment': 'Positive', 'confidence_score': ...",1
3,tx: 사랑합니다 [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다\nrx: 방금 전에 전화 ...,"{'sentiment': 'Positive', 'confidence_score': ...",1
4,rx: 금액은 어느 정도 하죠 그렇게 가격이 떨어져요.\ntx: 네네 이거는 뭐...,"{'sentiment': 'Neutral', 'confidence_score': 0...",1
5,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 안녕하세요.\...,"{'sentiment': 'Positive', 'confidence_score': ...",1
6,tx: 사랑합니다. [ㅇㅇㅇ] 성형외과 [ㅇㅇㅇ]입니다.\nrx: 아큐스컬프...,"{'sentiment': 'Positive', 'confidence_score': ...",1
7,tx: 네 안녕하세요 [ㅇㅇㅇ]님 맞으세요?\nrx: 여보세요? 네 네\nt...,"{'sentiment': 'Neutral', 'confidence_score': 0...",1
8,tx: 네 여보세요? 네네\nrx: 여보세요 네 검사실인데요.\ntx: 네네\...,"{'sentiment': 'Neutral', 'confidence_score': 0...",1
9,tx: 아 하는 경우들도 있거든요\nrx: 그거는 알고 계셔서 병원에서 하시는거...,"{'sentiment': 'Neutral', 'confidence_score': 0...",1


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GENERATE USING</strong>" 쿼리 구문을 사용하여 <strong>GPT</strong> 모델을 생성에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"task": 실행할 작업의 종류</li>
            <li>"text_col": 생성에 사용할 텍스트를 담은 컬럼의 이름</li>
            <li>"result_col": 데이터 테이블에서 생성 결과를 담을 컬럼 이름</li>
        </ul>
        </li>
    </ul>
</div>

## __7. 튜토리얼을 마치며__

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>생성형 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>